In [24]:
import torch

# Option 1: passing weights param as string
model = torch.hub.load("pytorch/vision", "resnet50", weights="IMAGENET1K_V2")

Using cache found in /Users/mikhail/.cache/torch/hub/pytorch_vision_main


In [25]:
import torchvision

model.eval()

# Trace the model with random data.
example_input = torch.rand(1, 3, 224, 224) 
traced_model = torch.jit.trace(model, example_input)
out = traced_model(example_input)

In [26]:
# Download class labels in ImageNetLabel.txt.
import urllib
label_url = 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt'
class_labels = urllib.request.urlopen(label_url).read().decode("utf-8").splitlines()
class_labels = class_labels[1:] # remove the first class which is background
assert len(class_labels) == 1000

In [27]:
# Set the image scale and bias for input image preprocessing.
import coremltools as ct
scale = 1/(0.226*255.0)
bias = [- 0.485/(0.229) , - 0.456/(0.224), - 0.406/(0.225)]

image_input = ct.ImageType(name="input_1",
                           shape=example_input.shape,
                           scale=scale, bias=bias)

In [28]:
# Using image_input in the inputs parameter:
# Convert to Core ML using the Unified Conversion API.
model = ct.convert(
    traced_model,
    inputs=[image_input],
    classifier_config = ct.ClassifierConfig(class_labels),
    compute_units=ct.ComputeUnit.CPU_ONLY,
)

Running MIL Clean up passes: 100%|███████████| 9/9 [00:00<00:00, 28.72 passes/s]
Translating MIL ==> NeuralNetwork Ops: 100%|█| 455/455 [00:03<00:00, 134.80 ops/


In [30]:
# Save the converted model.
model.save("resnet50.mlmodel")
# Print a confirmation message.
print('model converted and saved')

model converted and saved


In [32]:
# Load the test image and resize to 224, 224.
img_path = "panda.jpg"
img = PIL.Image.open(img_path)
img = img.resize([224, 224], PIL.Image.ANTIALIAS)

In [33]:
# Get the protobuf spec of the model.
spec = model.get_spec()
for out in spec.description.output:
    if out.type.WhichOneof('Type') == "dictionaryType":
        coreml_dict_name = out.name
        break

In [35]:
import numpy as np

# Make a prediction with the Core ML version of the model.
coreml_out_dict = model.predict({"input_1" : img})
print("coreml predictions: ")
print("top class label: ", coreml_out_dict["classLabel"])

coreml_prob_dict = coreml_out_dict[coreml_dict_name]

values_vector = np.array(list(coreml_prob_dict.values()))
keys_vector = list(coreml_prob_dict.keys())
top_3_indices_coreml = np.argsort(-values_vector)[:3]
for i in range(3):
    idx = top_3_indices_coreml[i]
    score_value = values_vector[idx]
    class_id = keys_vector[idx]
    print("class name: {}, raw score value: {}".format(class_id, score_value))

coreml predictions: 
top class label:  giant panda
class name: giant panda, raw score value: 6.912057399749756
class name: ram, raw score value: 1.781463623046875
class name: lesser panda, raw score value: 1.6441527605056763


### Make a PyTorch prediction and compare

In [36]:
# Make a prediction with the Torch version of the model:
# prepare the input numpy array.
img_np = np.asarray(img).astype(np.float32) # (224, 224, 3)
img_np = img_np[np.newaxis, :, :, :] # (1, 224, 224, 3)
img_np = np.transpose(img_np, [0, 3, 1, 2]) # (1, 3, 224, 224)
img_np = img_np / 255.0
torch_tensor_input = torch.from_numpy(img_np)

In [37]:
# Preprocess model for Torch.
transform_model = torch.nn.Sequential(
    torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                     std=(0.229, 0.224, 0.225)),
)

In [38]:
# Invoke prediction and print outputs.
torch_out = traced_model(transform_model(torch_tensor_input))

torch_out_np = torch_out.detach().numpy().squeeze()
top_3_indices = np.argsort(-torch_out_np)[:3]
print('torch top 3 predictions: ')
for i in range(3):
    idx = top_3_indices[i]
    score_value = torch_out_np[idx]
    class_id = class_labels[idx]
    print("class name: {}, raw score value: {}".format(class_id, score_value))

torch top 3 predictions: 
class name: giant panda, raw score value: 6.86653995513916
class name: ram, raw score value: 1.7364037036895752
class name: lesser panda, raw score value: 1.59848153591156


# Yolo

In [44]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/Users/mikhail/Projects/NNMALabs/NNMALab00.2/yolov5s.mlmodel')  # local model

Using cache found in /Users/mikhail/.cache/torch/hub/ultralytics_yolov5_master
INFO:yolov5:YOLOv5 🚀 2022-9-19 Python-3.9.12 torch-1.12.1 CPU

YOLOv5 🚀 2022-9-19 Python-3.9.12 torch-1.12.1 CPU

INFO:yolov5:Loading /Users/mikhail/Projects/NNMALabs/NNMALab00.2/yolov5s.mlmodel for CoreML inference...
Loading /Users/mikhail/Projects/NNMALabs/NNMALab00.2/yolov5s.mlmodel for CoreML inference...
INFO:yolov5:Adding AutoShape... 
Adding AutoShape... 
